# Library

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import os

# Get Embedding Model

In [ ]:
from embedded_utils import get_embedding_model
embedding_model = get_embedding_model()
llm = ?

INFO: Loading embedding model utility...
INFO: Initializing HuggingFaceEmbeddings model: all-MiniLM-L6-v2


/Users/faiisu/Documents/Project/RAGdevelop/src/embedded_utils.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/faiisu/Documents/Project/RAGdevelop/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
q_fin_dict = {
    'Risk Management': 'q-fin.RM',
    'Computational Finance': 'q-fin.CP',
    'Statistical Finance': 'q-fin.ST',
    'Trading and Market Microstructure': 'q-fin.TR',
    'Economics': 'q-fin.EC',
    'General Finance': 'q-fin.GN',
    'Mathematical Finance': 'q-fin.MF',
    'Portfolio Management': 'q-fin.PM',
    'Pricing of Securities': 'q-fin.PR'
}

# Get Vector

In [4]:
root_db_path = "../vectorDB"

In [7]:
persist_directory = 'q-fin.topic'
persist_directory = root_db_path + "/" + persist_directory
print(persist_directory)
print(f"Loading vector store from: {persist_directory}")

try:
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding_model
    )
    print("Vector store loaded successfully.")
    # Optional: Check the number of documents in the store
    print(f"Number of documents in store: {vectorstore._collection.count()}")
except Exception as e:
    print(f"Error loading vector store: {e}")
    # Handle error, maybe the directory doesn't exist or is corrupted
    exit()

../vectorDB/q-fin.topic
Loading vector store from: ../vectorDB/q-fin.topic
Vector store loaded successfully.
Number of documents in store: 9


# Retriver

In [8]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

print("Retriever created successfully.")

Retriever created successfully.


# Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """
You are an expert assistant for answering questions.
Use only the following context to answer the question at the end.
If you don't know the answer from the context, just say that you don't know.
Do not make up an answer.

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

prompt = ChatPromptTemplate.from_template(template)

# RAG Chain

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# --- 5. Build the RAG Chain using LCEL ---

# This function formats the retrieved documents into a single string
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# The core RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG chain created successfully. Ready to answer questions.")

# Chain runner

In [ ]:

# --- 6. Run the Chain ---
if __name__ == "__main__":
    while True:
        question = input("Ask a question (or type 'exit' to quit): ")
        if question.lower() == 'exit':
            break
        
        # Invoke the chain to get the answer
        answer = rag_chain.invoke(question)
        
        print("\n--- Answer ---\n")
        print(answer)
        print("\n--------------\n")